# Check csv file

- author : Sylvie Dagoret-Campagne
- creation date : 2025-09-15

In [ ]:
import pandas as pd

- holosummary_BG40_65mm_1~holo4_003.csv  
- holosummary_OG550_65mm_1~holo4_003.csv  
- holosummary_cyl_lens~holo4_001.csv  
- holosummary_empty~holo4_003.csv
- holosummary_BG40~holo4_003.csv         
- holosummary_SDSSr~holo4_003.csv         
- holosummary_cyl_lens~holo4_003.csv  
- holosummary_unknown~holo4_003.csv
- holosummary_FELH0600~holo4_003.csv     
- holosummary_collimator~holo4_003.csv    
- holosummary_empty~holo4_001.csv

In [ ]:
listexposures_file = "holosummary_empty~holo4_003.csv"

In [ ]:
df = pd.read_csv(listexposures_file,index_col=0)
df = df.reset_index(drop=True)

In [ ]:
df